In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

colors = ["#cccccc", "#FB9A99", "#B0DC89", "#33A02C", "#A6CEE3", "#1F78B4", "#FDBF6F", "#FF7F00"]

In [ ]:
def read_data(filename):
    column_names = ["Ciphers", "MaxConns", "ConcConns", "TransactionsPerSecond", "BytesPerTransaction", "TotalReadBytes", "TotalWriteBytes", "ReadTime", "ReadBandwidth", "WriteTime", "WriteBandwidth"]
    data = pd.read_csv(filename, names=column_names)
    return data

def compute_ts_mean_std(df):
    df = df[['Ciphers', 'ConcConns', 'TransactionsPerSecond']]
    return df.groupby(["Ciphers", "ConcConns"], as_index=False).agg([np.mean, np.std]).droplevel(axis=1, level=0).reset_index()   

In [ ]:
native = read_data('benchmark-native-large-pages.csv')
native_sgx = read_data('benchmark-native-sgx.csv')
wasm = read_data('benchmark-wasm.csv')
wasm_sgx = read_data('benchmark-wasm-sgx.csv')

In [ ]:
# Normalize the results based on REE time

## DISABLED

native_normalized = native # pd.DataFrame()
wasm_normalized = wasm # pd.DataFrame()
native_sgx_normalized = native_sgx # pd.DataFrame()
wasm_sgx_normalized = wasm_sgx # pd.DataFrame()
#for col in native:
#    if col == "Ciphers" or col == "ConcConns":
#        wasm_sgx_normalized[col] = wasm_sgx[col]
#        wasm_normalized[col] = wasm[col]
#        native_sgx_normalized[col] = native_sgx[col]
#        native_normalized[col] = native[col]
#    elif col == "TransactionsPerSecond":
#        wasm_sgx_normalized[col] = (wasm_sgx[col] / native[col])
#        wasm_normalized[col] = (wasm[col] / native[col])
#        native_sgx_normalized[col] = (native_sgx[col] / native[col])
#        native_normalized[col] = 1
    
#wasm_sgx_normalized

In [ ]:
native_grouped = compute_ts_mean_std(native_normalized)
native_sgx_grouped = compute_ts_mean_std(native_sgx_normalized)
wasm_grouped = compute_ts_mean_std(wasm_normalized)
wasm_sgx_grouped = compute_ts_mean_std(wasm_sgx_normalized)

native_sgx_grouped

In [ ]:
#
# Data visualization
#
def plot(ciphers, native, native_sgx, wasm, wasm_sgx):
    merged_aesgcm = pd.DataFrame()
    merged_aesgcm["ConcConns"] = native.loc[native["Ciphers"] == ciphers].reset_index(drop=True)["ConcConns"]
    merged_aesgcm["Native"] = native.loc[native["Ciphers"] == ciphers].reset_index(drop=True)["mean"]
    merged_aesgcm["NativeSgx"] = native_sgx.loc[native_sgx["Ciphers"] == ciphers].reset_index(drop=True)["mean"]
    merged_aesgcm["Wasm"] = wasm.loc[wasm["Ciphers"] == ciphers].reset_index(drop=True)["mean"]
    merged_aesgcm["WasmSgx"] = wasm_sgx.loc[wasm_sgx["Ciphers"] == ciphers].reset_index(drop=True)["mean"]
    merged_aesgcm.plot(x="ConcConns", title=ciphers, xlabel="Number of concurrents connections", ylabel="transactions per second")

In [ ]:
plot("TLS_AES_128_GCM_SHA256", native_grouped, native_sgx_grouped, wasm_grouped, wasm_sgx_grouped)

In [ ]:
plot("TLS_AES_128_CCM_SHA256", native_grouped, native_sgx_grouped, wasm_grouped, wasm_sgx_grouped)

In [ ]:
plot("TLS_CHACHA20_POLY1305_SHA256", native_grouped, native_sgx_grouped, wasm_grouped, wasm_sgx_grouped)

In [ ]:
#
# Export script
#
def export_file_by_ciphers(df, ciphers, filename):
    df = df.loc[df["Ciphers"] == ciphers].reset_index(drop=True)
    df.to_csv(filename, index=False)
    
for (ciphers, filename_prefix) in [("TLS_AES_128_GCM_SHA256", "aesgcm"), ("TLS_AES_128_CCM_SHA256", "aesccm"), ("TLS_CHACHA20_POLY1305_SHA256", "chachapoly")]:
    export_file_by_ciphers(native_grouped, ciphers, f"{filename_prefix}_native_formatted.csv")
    export_file_by_ciphers(native_sgx_grouped, ciphers, f"{filename_prefix}_native_sgx_formatted.csv")
    export_file_by_ciphers(wasm_grouped, ciphers, f"{filename_prefix}_wasm_formatted.csv")
    export_file_by_ciphers(wasm_sgx_grouped, ciphers, f"{filename_prefix}_wasm_sgx_formatted.csv")

In [ ]:
#
## Stats for the paper.
#

def r_ratio(value, number_of_digits = 1):
    return f"{round(value, number_of_digits)}".rstrip('0').rstrip('.')

native_average_transaction_per_sec = r_ratio(native_grouped["mean"].agg(np.mean), 0)
native_sgx_average_transaction_per_sec = r_ratio(native_sgx_grouped["mean"].agg(np.mean), 0)
wasm_average_transaction_per_sec = r_ratio(wasm_grouped["mean"].agg(np.mean), 0)
wasm_sgx_average_transaction_per_sec = r_ratio(wasm_sgx_grouped["mean"].agg(np.mean), 0)

native_sgx_average_ratio = r_ratio(native_grouped["mean"].agg(np.mean) / native_sgx_grouped["mean"].agg(np.mean))
wasm_average_ratio = r_ratio(native_grouped["mean"].agg(np.mean) / wasm_grouped["mean"].agg(np.mean))
wasm_sgx_sgx_average_ratio = r_ratio(native_grouped["mean"].agg(np.mean) / wasm_sgx_grouped["mean"].agg(np.mean))

native_sgx_vs_wasm_sgx_ratio = r_ratio(native_sgx_grouped["mean"].agg(np.mean) / wasm_sgx_grouped["mean"].agg(np.mean))

In [ ]:
#
## Export to LaTeX
#

f = open("tls-export.tex", "w")

f.write(f"\\def\\tlsAverageNative{{{native_average_transaction_per_sec}}}\n")
f.write(f"\\def\\tlsAverageSgx{{{native_sgx_average_transaction_per_sec}}}\n")
f.write(f"\\def\\tlsAverageWasm{{{wasm_average_transaction_per_sec}}}\n")
f.write(f"\\def\\tlsAverageTwine{{{wasm_sgx_average_transaction_per_sec}}}\n")

f.write(f"\\def\\tlsRatioSgx{{{native_sgx_average_ratio}}}\n")
f.write(f"\\def\\tlsRatioWasm{{{wasm_average_ratio}}}\n")
f.write(f"\\def\\tlsRatioTwine{{{wasm_sgx_sgx_average_ratio}}}\n")
f.write(f"\\def\\tlsRatioSgxVsTwine{{{native_sgx_vs_wasm_sgx_ratio}}}\n")

f.close()